In [1]:
import sacct
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
import dask.dataframe as dd
from random import randint
import numpy as np
from dask.array import from_array as fa

sourceFile = 'df_sacct.parquet'  # Source of sacct data if using offline mode.

In [2]:
#df = dd.read_csv(sourceFile)
df = dd.read_parquet(sourceFile)  # For if you're smart and use parquet files instead of csvs

In [3]:
pd.set_option('display.max.columns', None)
pd.set_option('display.width', 150)
print(df.shape,'\n\n')
print(df.head(),'\n\n')
df.describe()

(Delayed('int-0434419b-6b7c-46ea-928e-4aeb9dcae011'), 77) 


   Unnamed: 0  Unnamed: 0.1  JobIDRaw   step           JobID  UID  GID  AssocID Cluster                 JobName  User  Group Account  Reservation  \
0           0             0  35767092  batch  35767092.batch  NaN  NaN    24786   comet                   batch   NaN    NaN  mit178          NaN   
1           1             1  35767116  batch  35767116.batch  NaN  NaN    24786   comet                   batch   NaN    NaN  mit178          NaN   
2           2             2  35790946      0      35790946.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN   
3           3             3  35790948      0      35790948.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN   
4           4             4  35790951      0      35790951.0  NaN  NaN     1263   comet  _batch_command.cmdline   NaN    NaN  sds121          NaN   

   ReservationId  Partition  QOS  QOSRAW  NN

,Unnamed: 0,Unnamed: 0.1,JobIDRaw,UID,GID,AssocID,User,Group,Reservation,ReservationId,Partition,QOS,QOSRAW,NNODES,NTASKS,NCPUS,AllocCPUS,ReqCPUS,Timelimit,Priority,DerivedExitCode,Time,MinCPUTask,ResvCPURaw,CPUTimeRaw,MaxDiskReadTask,MaxDiskWriteTask,MaxPagesTask,MaxRSSTask,MaxVMSizeTask,ConsumedEnergy,ConsumedEnergyRaw,Comment
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
# Example of populating empty column with new data

runTimes = df['CPUTimeRaw'].compute().tolist()

limitExamples = []
for time in runTimes:
    newTime = round((time+randint(0,100)/10)*(1+randint(0,100)/100))
    limitExamples.append(newTime)

print(runTimes[:10])
print(limitExamples[:10])
df['Timelimit'] = fa(limitExamples)
print(df['Timelimit'])
dd.to_csv(df,'df_sacctFix.csv')

[24, 24, 8, 12, 6, 12, 6, 8, 24, 24]
[33, 40, 14, 23, 16, 18, 17, 15, 44, 58]
Dask Series Structure:
npartitions=1
    int64
      ...
Name: Timelimit, dtype: int64
Dask Name: getitem, 7 tasks


['/oasis/scratch/comet/dougfe/temp_project/projects/sacctNotebook/cometnb/df_sacctFix.csv/0.part']

In [5]:
# Time wasted query
timeWaste = []
timeEff = []

timeLimit = df['Timelimit'].compute().tolist()
cpuTime = df['CPUTimeRaw'].compute().tolist()
    #timeEff.append(round(100*(value/df['Timelimit'].compute().tolist()[ind])))


In [6]:
for ind, value in enumerate(cpuTime):
    timeWaste.append(timeLimit[ind] - value)

In [ ]:
print(df['CPUTimeRaw'].compute().tolist()[:10])
print(df['Timelimit'].compute().tolist()[:10])
print((timeWaste)[:10])

In [ ]:
# Load in an old query rather than build a new one
queryDF = pd.read_parquet('sacctQuery.parquet')

In [ ]:
# Construct dictionary and pandas df with old and new data to display query
#newCatDir = {'CPUTimeRaw':df['CPUTimeRaw'].compute().tolist(), 'TimeLimit':df['Timelimit'].compute().tolist(), 'TimeWaste': timeWaste}
newCatDir = {'JobIDRaw': df['JobIDRaw'],'CPUTimeRaw': df['CPUTimeRaw'], 'TimeLimit': df['Timelimit'], 'TimeWaste': timeWaste}#, 'TimeEff%': timeEff}

queryDF = pd.DataFrame(newCatDir)

In [ ]:
# Display query dataframe preview
queryDF.head(50)

In [ ]:
queryDF.describe()

In [ ]:
# Sorting query df
queryDF.sort_values('TimeWaste', ascending=False)

In [ ]:
# Query save system: If desired, users can save their current query as a parquet file for use later

saveName = 'sacctQuery'  # Doesn't need to include the .parqet or .csv extension
queryDF.to_parquet(saveName+'.parquet')
